<a href="https://colab.research.google.com/github/Abbas-Muneer/Olympians/blob/DataScience/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==2.7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.6/489.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.1/463.1 KB 26.1 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.11.0
    Uninstalling tensorflow-estimator-2.11.0:
      Successfully uninstalled tensorflow-estimator-2.11.0
  Attempting uninstall: keras
    Found existing installation: keras 2.11.0
    Uninstalling keras-2.11.0:
      Successfully uninstalled keras-2.11.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 23.1.21
    Uninstalling flatbuffers-23.1.21:
      Successfully uninstalled flatbuffers-23.1.21
  Attempting uninstall: tensorflow
    F

In [3]:
import os
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2,preprocess_input
from tensorflow.keras.layers import Dense,Conv2D,GlobalAveragePooling2D,Input
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras import callbacks,optimizers
import numpy as np
from google.colab import drive

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd drive/MyDrive/model_training/


/content/drive/MyDrive/model_training


In [5]:
for i in os.listdir("content"):
  print(i)

resnet_model_training
resnet-model-23-02-21.h5
3rdModel.h5
Copy of resnet-model-23-02-21.h5


In [ ]:
!ls raw-img/

 ACNE		     Basal_Cell_Carcinoma   Melanocytic_Nevi   Psoriasis
 Atopic_Dermatitis   Eczema		    Melanoma	      'Skin Cancer'


In [ ]:
for i in os.listdir("raw-img"):
  print(i,len(os.listdir("raw-img/" + i)))

ACNE 375
Skin Cancer 3543
Psoriasis 341
Basal_Cell_Carcinoma 1323
Melanocytic_Nevi 2020
Atopic_Dermatitis 481
Melanoma 1140
Eczema 18


In [ ]:
try:
  os.mkdir("train")
  os.mkdir("test")
except:
  pass  
for i in os.listdir("raw-img"):
  try:
    os.mkdir("train/" + i)
    os.mkdir("test/" + i)
  except:
    pass  
  for j in os.listdir("raw-img/" + i)[:1600]:
    os.rename("raw-img/"+i+"/"+j, "train/"+i+"/"+j)
  for j in os.listdir("raw-img/" + i)[:400]:
    os.rename("raw-img/"+i+"/"+j, "test/"+i+"/"+j)

In [ ]:
for i in os.listdir("train"):
  print(i,len(os.listdir("train/" + i)))

ACNE 1600
Psoriasis 1600
Basal_Cell_Carcinoma 1600
Melanocytic_Nevi 1600
Atopic_Dermatitis 1600
Melanoma 1600
Eczema 1600
Skin_Cancer 1600
Warts 1600
Tinea Ringworm Candideas 1600
Serorrheic Keratoses 1600
Nail Fungus 1600


In [6]:
def img_Data(dir_path,target_size,batch,class_lst,preporcssing):
  if preporcssing:
    gen_object = ImageDataGenerator(preprocessing_function=preporcssing)
  else:
    gen_object = ImageDataGenerator()

  return(gen_object.flow_from_directory(dir_path,
                                        target_size=target_size,
                                        batch_size=batch,
                                        class_mode='sparse',
                                        classes = class_lst,
                                        shuffle=True))

In [7]:
train_data_gen = img_Data("train",(224,224),500,os.listdir("train"),preprocess_input)
valid_data_gen = img_Data("test",(224,224),500,os.listdir("test"),preprocess_input)

Found 19199 images belonging to 12 classes.
Found 4800 images belonging to 12 classes.


In [8]:
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2( input_shape=(224,224,3),
                                                             alpha=1.0,
                                                             include_top=False,
                                                             weights='imagenet',
                                                             input_tensor=None,
                                                             pooling=None,
                                                             classes=1000,
                                                             classifier_activation='softmax')
                                        
   


9420800/9406464 [==============================] - 0s 0us/step


In [9]:
base_model.trainable = False

In [10]:
model = tf.keras.models.Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024,activation='relu'))
model.add(Dense(8,activation='softmax'))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
              

In [11]:
elst = callbacks.EarlyStopping(monitor='val_loss',patience=5,mode='min')
save_ck = callbacks.ModelCheckpoint(' .mdl_wt.hdf5',save_best_only=True,monitor='val_loss',mode= 'min')

In [ ]:
model.fit(train_data_gen,batch_size=500,validation_data=valid_data_gen,callbacks=[elst,save_ck],epochs=10)

In [2]:
#retrying model training with resnet model instead of mobileNetV2 model
!pip install tensorflow keras-tuner opencv-python matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 KB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.3 MB/s eta 0:00:00


In [6]:
import numpy as np
from matplotlib import pyplot as plt

In [7]:
DATA_TRAINING_PATH = 'train'
BATCH_SIZE = 32
IMG_HEIGHT, IMG_WIDTH = 224, 224
IMG_SIZE = (IMG_HEIGHT, IMG_WIDTH, 3)

DATA_TESTING_PATH = 'test'
BATCH_SIZE = 32
IMG_HEIGHT, IMG_WIDTH = 224, 224
IMG_SIZE = (IMG_HEIGHT, IMG_WIDTH, 3)

In [8]:
train_data = tf.keras.utils.image_dataset_from_directory(
    DATA_TRAINING_PATH,
    label_mode='categorical',
    color_mode='rgb',
    validation_split=0.0001,
    subset="training",
    seed=42,
    shuffle=True,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
    )

val_data = tf.keras.utils.image_dataset_from_directory(
    DATA_TESTING_PATH,
    label_mode='categorical',
    color_mode='rgb',
    validation_split=0.9999,
    subset="validation",
    seed=42,
    shuffle=False,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
    )

Found 19199 files belonging to 12 classes.
Using 19198 files for training.
Found 4800 files belonging to 12 classes.
Using 4799 files for validation.


In [9]:
class_names = train_data.class_names

# optimize
train_data = train_data.prefetch(tf.data.AUTOTUNE)
val_data = val_data.prefetch(tf.data.AUTOTUNE)

In [10]:
from tensorflow import keras


In [11]:
pip install resnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for resnet: filename=resnet-0.1-py3-none-any.whl size=10044 sha256=ca6fd6947373dd23ab5a11019d8592ea1ba681fb9ba4a8a7c9b8e7b9c1fe43c0
  Stored in directory: /root/.cache/pip/wheels/3c/db/ed/d285a3be8fa331aca2474dbd87b0e6c2ce5a9060f836723464
Successfully built resnet


In [12]:
from tensorflow.keras.applications import ResNet50V2

In [13]:
base_model = ResNet50V2(
    include_top=False, # Exclude ImageNet classifier at the top
    weights='imagenet',
    input_shape=IMG_SIZE
    )



# Freeze the base_model
base_model.trainable = False

94668760/94668760 [==============================] - 1s 0us/step


In [14]:
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomTranslation, RandomContrast

In [15]:
data_augmentation = keras.Sequential(
    [RandomFlip('horizontal'), 
     RandomRotation(factor=(-0.025, 0.025)),
     RandomTranslation(height_factor=0.1, width_factor=0.1),
     RandomContrast(factor=0.1)
     ])



# Create new model on top
inputs = keras.Input(shape=IMG_SIZE)
x = data_augmentation(inputs)

x = keras.applications.resnet_v2.preprocess_input(x)
x = base_model(x, training=False)

In [16]:
from tensorflow.keras.layers import GlobalAveragePooling2D, BatchNormalization, Dropout, Flatten, Dense

In [17]:
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)  # Regularize with dropout

outputs = Dense(len(class_names), activation="softmax", name="pred")(x)

resnet_model = keras.Model(inputs, outputs)



resnet_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 224, 224, 3)      0         
 a)                                                              
                                                                 
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0     

In [18]:
from tensorflow.keras.metrics import *
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

In [26]:
metrics = [CategoricalAccuracy(), Precision(), Recall()]

resnet_model.compile(
    loss = CategoricalCrossentropy(),
    optimizer = Adam(learning_rate=0.001),
    metrics = metrics
    )

elst = callbacks.EarlyStopping(monitor='val_loss',patience=25,mode='min')
save_ck = callbacks.ModelCheckpoint('resnet-model-23-02-21.h5',save_best_only=True,monitor='val_loss',mode= 'min')


resnet_history = resnet_model.fit(
      train_data, 
      epochs=25, 
      validation_data=val_data,
      callbacks=[elst],
      verbose=1
      )



resnet_model.save('AbbasResnet3Model.h5')



result = resnet_model.evaluate(val_data)
result




Epoch 1/25
600/600 [==============================] - 2755s 4s/step - loss: 1.4283 - categorical_accuracy: 0.5243 - precision: 0.6427 - recall: 0.4204 - val_loss: 1.3392 - val_categorical_accuracy: 0.5443 - val_precision: 0.6543 - val_recall: 0.4409
Epoch 2/25
600/600 [==============================] - 147s 242ms/step - loss: 1.1611 - categorical_accuracy: 0.6004 - precision: 0.7012 - recall: 0.5066 - val_loss: 1.2613 - val_categorical_accuracy: 0.5624 - val_precision: 0.6707 - val_recall: 0.4682
Epoch 3/25
600/600 [==============================] - 134s 222ms/step - loss: 1.0979 - categorical_accuracy: 0.6142 - precision: 0.7107 - recall: 0.5283 - val_loss: 1.2042 - val_categorical_accuracy: 0.5760 - val_precision: 0.6909 - val_recall: 0.4797
Epoch 4/25
600/600 [==============================] - 134s 222ms/step - loss: 1.0620 - categorical_accuracy: 0.6239 - precision: 0.7201 - recall: 0.5410 - val_loss: 1.2149 - val_categorical_accuracy: 0.5868 - val_precision: 0.6891 - val_recall: 0

/usr/local/lib/python3.8/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


150/150 [==============================] - 24s 163ms/step - loss: 1.1201 - categorical_accuracy: 0.6051 - precision: 0.7067 - recall: 0.5201


[1.1201198101043701,
 0.6051260828971863,
 0.7066817879676819,
 0.5201083421707153]

In [ ]:
infection_CLASSES = {x:y for x,y in enumerate(class_names)}
MODEL_DIR = 'My Drive/model_training/content/AbbasResnet3Model.h5'
MODEL = keras.models.load_model('content/AbbasResnet3Model.h5')

def predict_image_class(img_path:str) -> str:
  img = keras.preprocessing.image.load_img(img_path, target_size=IMG_SIZE)
  img_array = keras.preprocessing.image.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0)  # Create batch axis

  predictions = MODEL.predict(img_array)
  print(predictions.argmax(axis=1))

  return infection_CLASSES[predictions.argmax(axis=1)[0]]



TEST_IMG_DIR = "content/resnet_model_training/Nail_Fungus.jpg"
print(f"Prediction - {predict_image_class(TEST_IMG_DIR)}")



from google.colab import files
files.download('content/AbbasResnet3Model.h5')